# Extracting polygons
This function uses the excel file created for Quad-G+ to crop the maps to the correct coordinates after its extracted the boundaries from the geotiff files and saved it as a shapefile.

In [1]:
# Import libraries
import pandas as pd
import os
import rasterio
import numpy as np
from rasterio.mask import mask
from shapely.geometry import box, Polygon
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from skimage import morphology, feature, measure


In [ ]:
# Takes DMS coordinate strings and converts them to decimal degrees
# Handles formats like "35 45" or "-120 07 30" that are found in excel file
def parse_dms(dms_str):
    # Remove any quotes
    dms_str = str(dms_str).strip('"\'')
    
    # Split into parts
    parts = dms_str.split()
    
    # Set defaults
    degrees = 0
    minutes = 0
    seconds = 0
    
    # Parse
    if len(parts) >= 1:
        degrees = float(parts[0])
    if len(parts) >= 2:
        minutes = float(parts[1])
    if len(parts) >= 3:
        seconds = float(parts[2])
    
    # Convert
    decimal = abs(degrees) + minutes/60 + seconds/3600
    return decimal if degrees >= 0 else -decimal

# Main function for field boundary extraction
# Takes a GeoTIFF and returns a shapefile with field boundaries
def extract_field_boundaries(geotiff_path, output_path=None):
    with rasterio.open(geotiff_path) as src:
        image = src.read()
        transform = src.transform
        crs = src.crs
        
        # Grayscale
        gray_image = np.mean(image, axis=0)
        
        # Boost the contrast
        p2, p98 = np.percentile(gray_image, (2, 98))
        gray_image = np.interp(gray_image, (p2, p98), (0, 1))
        
        # Convert to black and white
        binary = gray_image < 0.45
        
        # Get rid of tiny specks
        cleaned = morphology.remove_small_objects(binary, min_size=200)
        
        # Clean up
        cleaned = morphology.binary_opening(cleaned, morphology.square(2))
        cleaned = morphology.binary_closing(cleaned, morphology.square(4))
        
        # Find edges
        edges = feature.canny(
            cleaned,
            sigma=1.8,
            low_threshold=0.05,
            high_threshold=0.25
        )
        
        # Make edges thicker and fill gaps
        edges = morphology.dilation(edges, morphology.square(2))
        edges = morphology.closing(edges, morphology.square(5))
        
        # Clean up noise again
        edges = morphology.remove_small_objects(edges, min_size=200)
        
        # Find lines in the edge image
        contours = measure.find_contours(edges, 0.1)
        
        # Turn lines into actual polygons
        polygons = []
        for contour in contours:
            if len(contour) > 8:  # Skip tiny contours
                start = contour[0]
                end = contour[-1]
                dist = np.linalg.norm(start - end)
                
                # Close polygon if not already closed
                if dist < 30: 
                    contour = np.vstack([contour, contour[0]])
                
                # Convert coordinates
                geo_coords = [rasterio.transform.xy(transform, coord[0], coord[1]) 
                            for coord in contour]
                
                try:
                    poly = Polygon(geo_coords)
                    # Only keep reasonable sized polygons
                    if poly.is_valid and 800 < poly.area < 3e6: 
                        poly = poly.simplify(1.0, preserve_topology=True)
                        if poly.is_valid:
                            polygons.append(poly)
                except:
                    continue
        
        # Put into GeoDataFrame
        gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)
        
        # Remove big polygons (quad borders)
        large_polys = gdf[gdf.area > 1e6]
        field_polys = gdf[gdf.area <= 1e6]
        
        # Remove polygons inside other polygons (field text)
        not_contained = []
        for idx, row in field_polys.iterrows():
            poly1 = row.geometry
            is_contained = False
            for idx2, row2 in field_polys.iterrows():
                if idx != idx2:
                    poly2 = row2.geometry
                    if poly2.contains(poly1):
                        is_contained = True
                        break
            if not is_contained:
                not_contained.append(poly1)
        
        # Put the quad polygons back with text filtered out
        all_polygons = not_contained + large_polys.geometry.tolist()
        
        # New gdf with cleaned polygons
        gdf = gpd.GeoDataFrame(geometry=all_polygons, crs=crs)
        
        # Smooth out polygons
        gdf = gdf.buffer(0.8).buffer(-0.8)  
        gdf = gpd.GeoDataFrame(geometry=gdf, crs=crs)
        
        # Save
        if output_path:
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            gdf.to_file(output_path)
        
        return gdf

# Plot the shapefile
def plot_with_coordinates(shapefile_path):
    gdf = gpd.read_file(shapefile_path)
    
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Convert coordinates
    if gdf.crs != 'EPSG:3857':
        gdf = gdf.to_crs(epsg=3857)
    
    gdf.plot(ax=ax, 
            edgecolor='red',
            facecolor='none',
            linewidth=0.5)
    
    ax.grid(True)
    ax.tick_params(labelsize=8)
    
    return fig, ax

# Crops GeoTIFF to coordinates from excel then extracts field boundaries
def crop_and_extract(input_path, output_tif_path, output_shp_path, bounds):
    # Check file actually exists
    if not os.path.exists(input_path):
        print(f"Input file not found: {input_path}")
        return None
        
    # Open source file and print coordinate info
    with rasterio.open(input_path) as src:
        print(f"\nProcessing {os.path.basename(input_path)}:")
        print("Original image bounds:", src.bounds)
        print("Original image CRS:", src.crs)
        
        # Create bounding box
        bbox = box(bounds['west'], bounds['south'], bounds['east'], bounds['north'])
        bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs='EPSG:4326')  # WGS84
        
        # Transform bbox to match crs
        bbox_gdf = bbox_gdf.to_crs(src.crs)
        bbox_transformed = bbox_gdf.geometry.values[0]
        
        print("Transformed crop bounds:", bbox_transformed.bounds)
        
        try:
            # Crop
            out_image, out_transform = mask(src, [bbox_transformed], crop=True)
            
            print("Cropped image shape:", out_image.shape)
            
            # Update metadata
            out_meta = src.meta.copy()
            out_meta.update({
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            
            # Output directory
            os.makedirs(os.path.dirname(output_tif_path), exist_ok=True)
            
            # Save cropped image
            with rasterio.open(output_tif_path, "w", **out_meta) as dest:
                dest.write(out_image)
        
        except ValueError as e:
            print(f"Error during cropping: {e}")
            print("Check if the coordinates are within the image bounds and in the correct order.")
            return None
    
    # Extract field boundaries from cropped image
    try:
        gdf = extract_field_boundaries(output_tif_path, output_shp_path)
        return gdf
    except Exception as e:
        print(f"Error during field boundary extraction: {e}")
        return None

# Process GeoTIFF files from the excel sheet
# Has resume capability so you don't have to start over if something goes wrong (processing_results.csv)
def batch_process_geotiffs(excel_path, input_dir=".", output_dir="outputs", resume=True):
    # Make output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Check csv for already processed files
    results_path = os.path.join(output_dir, 'processing_results.csv')
    existing_results = []
    processed_files = set()
    
    if resume and os.path.exists(results_path):
        try:
            results_df = pd.read_csv(results_path)
            existing_results = results_df.to_dict('records')
            
            for result in existing_results:
                if result['status'] == 'Success':
                    processed_files.add(result['file_name'])
            
            print(f"Found {len(processed_files)} previously processed files.")
        except Exception as e:
            print(f"Error reading existing results file: {e}")
    
    # Load excel file
    try:
        df = pd.read_excel(excel_path)
        print(f"Read {len(df)} entries from {excel_path}")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return
    
    # Check columns
    required_columns = ["File Name", "South", "North", "West", "East"]
    
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"Missing required columns in Excel file: {missing_columns}")
        return
    
    # Check if resuming
    results = existing_results.copy() if resume else []
    
    # Keep track how many processed
    processed_count = 0
    
    # Process rows excel file
    for idx, row in df.iterrows():
        file_name = row['File Name']
        
        # Skip if in csv already
        if resume and file_name in processed_files:
            print(f"Skipping already processed file: {file_name}")
            continue
            
        print(f"\n{'='*50}")
        print(f"Processing entry {idx+1}/{len(df)}: {file_name}")
        
        # File paths
        input_file = os.path.join(input_dir, file_name)
        base_name = Path(file_name).stem
        
        # Convert coordinates
        try:
            south = parse_dms(str(row['South']))
            north = parse_dms(str(row['North']))
            west = parse_dms(str(row['West']))
            east = parse_dms(str(row['East']))
            
            print("\nConverted coordinates:")
            print(f"South: {south}")
            print(f"North: {north}")
            print(f"West: {west}")
            print(f"East: {east}")
            
            # Output file paths
            output_tif_path = os.path.join(output_dir, f"{base_name}_cropped.tif")
            output_shp_path = os.path.join(output_dir, f"{base_name}_cropped.shp")
            output_png_path = os.path.join(output_dir, f"{base_name}_cropped.png")
            
            # Bounds
            bounds = {
                'south': south,
                'north': north,
                'west': west,
                'east': east
            }
            
            # Process
            gdf = crop_and_extract(input_file, output_tif_path, output_shp_path, bounds)
            
            status = "Success" if gdf is not None else "Failed"
            
            # Plot
            if gdf is not None:
                fig, ax = plot_with_coordinates(output_shp_path)
                plt.savefig(output_png_path, dpi=300, bbox_inches='tight')
                plt.close()
            
            # Record in csv
            results.append({
                'file_name': file_name,
                'status': status,
                'output_tif': output_tif_path if gdf is not None else None,
                'output_shp': output_shp_path if gdf is not None else None,
                'output_png': output_png_path if gdf is not None else None
            })
            
            processed_count += 1
            
            print(f"Processing status: {status}")
            
            # Save csv after each file
            results_df = pd.DataFrame(results)
            results_df.to_csv(results_path, index=False)
            
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
            results.append({
                'file_name': file_name,
                'status': 'Error',
                'error_message': str(e),
                'output_tif': None,
                'output_shp': None,
                'output_png': None
            })
            # Save progress after errors
            results_df = pd.DataFrame(results)
            results_df.to_csv(results_path, index=False)
    
    # Print summary
    print(f"\n{'='*50}")
    print("Processing Summary:")
    success_count = sum(1 for r in results if r['status'] == 'Success')
    print(f"Total files in Excel: {len(df)}")
    print(f"Previously processed: {len(processed_files)}")
    print(f"Newly processed this run: {processed_count}")
    print(f"Total successful: {success_count}")
    print(f"Total failed: {len(results) - success_count}")
    
    # Save
    results_df = pd.DataFrame(results)
    results_df.to_csv(results_path, index=False)
    print(f"Summary saved to: {results_path}")
    
    return results_df

# Visualize shapefiles
def plot_all_shapefiles(shapefile_dir, output_path=None, base_crs='EPSG:3857'):
    shapefile_paths = list(Path(shapefile_dir).glob("*.shp"))
    
    if not shapefile_paths:
        print(f"No shapefiles found in {shapefile_dir}")
        return None, None
    
    print(f"Found {len(shapefile_paths)} shapefiles")
    
    all_gdfs = []
    
    for shp_path in shapefile_paths:
        try:
            gdf = gpd.read_file(shp_path)
            
            filename = shp_path.stem
            gdf['source_file'] = filename
            
            # Check crs
            if gdf.crs != base_crs:
                gdf = gdf.to_crs(base_crs)
                
            all_gdfs.append(gdf)
            print(f"Loaded: {filename}")
        except Exception as e:
            print(f"Error loading {shp_path}: {e}")
    
    if not all_gdfs:
        print("No valid shapefiles could be loaded")
        return None, None
    
    # Combine files
    combined_gdf = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True))
    
    # Plot
    fig, ax = plt.subplots(figsize=(20, 20))
    combined_gdf.plot(ax=ax, edgecolor='blue', facecolor='none', linewidth=0.8)
    ax.grid(True, linestyle='--', alpha=0.5)
    
    ax.set_title(f"Combined Field Boundaries", fontsize=14)

    
    # Save
    if output_path:
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Combined visualization saved to: {output_path}")
    
    return fig, ax


In [2]:
excel_path = "excel_map_info/quad_maps_1969.xlsx"  # Path to your Excel file
input_dir = "quad_maps/maps_1969"                # Directory where your GeoTIFF files are located
output_dir = "outputs/outputs_1969"         # Directory to save outputs
    
    
# Run the batch processing
results = batch_process_geotiffs(excel_path, input_dir, output_dir, resume=True)


Read 52 entries from excel_map_info/quad_maps_1969.xlsx

Processing entry 1/52: 69KN5036_Poly.tif

Converted coordinates:
South: 35.75
North: 35.875
West: -120.125
East: -120.0

Processing 69KN5036_Poly.tif:
Original image bounds: BoundingBox(left=-6845.491678735414, bottom=-1337.9296014289866, right=7166.130376689578, top=15735.97613588989)
Original image CRS: PROJCS["unnamed",GEOGCS["NAD27",DATUM["North_American_Datum_1927",SPHEROID["Clarke 1866",6378206.4,294.978698213898,AUTHORITY["EPSG","7008"]],AUTHORITY["EPSG","6267"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4267"]],PROJECTION["Polyconic"],PARAMETER["latitude_of_origin",35.75],PARAMETER["central_meridian",-120.0625],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Transformed crop bounds: (-5564.870126976284, 5.778656491383042, 5740.901211980603, 13875.6627628585)
Cropped imag